In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from scipy.stats import mode
import operator
import functools
import datetime 

# Read input
df1 = pd.read_csv('datasets/BPI Challenge 2017-training.csv')
df2 = pd.read_csv('datasets/BPI Challenge 2017-test.csv')
frames = [df1, df2]
df = pd.concat(frames)
df.reset_index(drop=True, inplace=True)


# Parse the timestamp and convert it into y-m-d form
df['event time:timestamp'] = pd.to_datetime(df['event time:timestamp'], format = '%d-%m-%Y %H:%M:%S.%f')

# Sort data by timestamp in ascending order
df.sort_values(['event time:timestamp'], axis=0, inplace=True)
    
# split into train set and test set (80/20)
df_train, df_test = train_test_split(df, test_size=0.2, shuffle = False)
# remove cases started in the training set
df_test = df_test[~df_test['case concept:name'].isin(df_train['case concept:name'].values)]

# Reset index
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Assign position number to each event
df_sort = df_train.set_index(df_train.groupby('case concept:name').cumcount(), append = True)


In [8]:
#calculate time since started for each case
df_sort = df_sort.assign(time_since_started=df_sort.groupby('case concept:name')['event time:timestamp'].apply(lambda x: x - x.iloc[0]))
#groupby case concept and calculate average for each position
avg_timespan = df_sort.groupby(level=1)['time_since_started'].apply(
    lambda x: x.astype('timedelta64[s]').mean()
)
avg_timespan = avg_timespan.rename({'time_since_started': 'avg_time_since_started'})

# merge
res = pd.DataFrame()
res = df_sort.merge(avg_timespan, left_on = df_sort.index.get_level_values(1).values, right_index = True, how = 'left')


In [9]:
res['time_since_started_y'] + res['event time:timestamp']

TypeError: unsupported operand type(s) for +: 'numpy.ndarray' and 'DatetimeArray'

In [10]:
res

,,eventID,case LoanGoal,case ApplicationType,case concept:name,case RequestedAmount,event Action,event org:resource,event concept:name,event EventOrigin,event EventID,event lifecycle:transition,event time:timestamp,time_since_started_x,time_since_started_y
0,0,0,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 10:51:15.304,0 days 00:00:00,0.000000e+00
1,1,1,Existing loan takeover,New credit,Application_652823628,20000.0,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 10:51:15.352,0 days 00:00:00.048000,1.324529e-02
2,2,2,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 10:51:15.774,0 days 00:00:00.470000,8.983760e-02
3,3,3,Existing loan takeover,New credit,Application_652823628,20000.0,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 10:52:36.392,0 days 00:01:21.088000,2.474544e+03
4,4,4,Existing loan takeover,New credit,Application_652823628,20000.0,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 10:52:36.403,0 days 00:01:21.099000,2.576444e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961808,18,110732846825490,Existing loan takeover,New credit,Application_2048381730,30000.0,Released,User_2,W_Call after offers,Workflow,Workitem_458551822,suspend,2016-10-27 11:13:41.439,2 days 11:37:10.912000,6.441544e+05
961809,32,108142981546016,Remaining debt home,New credit,Application_1385143613,25000.0,Released,User_14,W_Call incomplete files,Workflow,Workitem_816570513,suspend,2016-10-27 11:14:18.490,8 days 15:21:50.032000,1.271193e+06
961810,24,109474421407768,Home improvement,New credit,Application_1273835556,60000.0,Obtained,User_14,W_Call incomplete files,Workflow,Workitem_1843181301,resume,2016-10-27 11:14:22.469,5 days 20:25:41.846000,1.085563e+06
961811,31,108181636251679,Car,New credit,Application_259502445,6000.0,Deleted,User_118,W_Call incomplete files,Workflow,Workitem_2066573185,ate_abort,2016-10-27 11:15:07.489,8 days 14:15:26.804000,1.244574e+06


In [ ]:
#res['time_since_started_y']=res['time_since_started_y'].astype('timedelta64[ns]')